**COVID-19 DETECTION FROM CHEST X-RAYS USING Densenet-121 model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Covid-detection

/content/drive/MyDrive/Colab Notebooks/Covid-detection


In [ ]:
import numpy as np
import os
import cv2 as cv
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.optimizers import SGD

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Covid-detection/COVID-19_Radiography_Dataset/'

sets = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']

x, y = ([], [])

# Loop through all categories
for s in sets:
    path_to_cat = os.path.join(path, s)

    count=0;
    for i in os.listdir(path_to_cat):
        if s=='COVID':
           if count==2000:
              break;
        else:
          if count==700:
              break

        count+=1

        # Path to image
        image_path = os.path.join(path_to_cat, i)
        
        image = cv.imread(image_path)
        image = cv.resize(image, (224, 224)).reshape(-1, 224, 224, 3)
      
        x.append(image)
        
        if s=='COVID':
            y.append(0)
        else:
            y.append(1)

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
#Train set 80% Test 15% Validation 10%

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42,stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test,y_test,test_size=0.25,random_state=42,stratify=y_test)

In [ ]:
print("Training samples:",X_train.shape[0])
print("Testing samples:",X_test.shape[0])
print("Validation samples:",X_val.shape[0])

Training samples: 3280
Testing samples: 615
Validation samples 205


In [ ]:
datagenerator = ImageDataGenerator(rescale = 1./255.)

In [ ]:
img_size=224
X_train = X_train.reshape(-1, img_size, img_size, 3)
X_val = X_val.reshape(-1, img_size, img_size, 3)
X_test = X_test.reshape(-1, img_size, img_size, 3)

In [ ]:
datagenerator.fit(X_train)

In [ ]:
from keras.applications.densenet import DenseNet121
densenet = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3))

In [ ]:
for layer in densenet.layers:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(densenet)
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = SGD(learning_rate=0.001, momentum=0.9, name="SGD"),metrics = ['acc'])

history = model.fit(datagenerator.flow(X_train,y_train, batch_size = 32) ,epochs = 10 ,validation_data = datagenerator.flow(X_val, y_val))

Epoch 1/10
103/103 [==============================] - 65s 271ms/step - loss: 0.5440 - acc: 0.7621 - val_loss: 0.1936 - val_acc: 0.9317
Epoch 2/10
103/103 [==============================] - 23s 228ms/step - loss: 0.1964 - acc: 0.9218 - val_loss: 0.1362 - val_acc: 0.9561
Epoch 3/10
103/103 [==============================] - 23s 228ms/step - loss: 0.1256 - acc: 0.9478 - val_loss: 0.1435 - val_acc: 0.9610
Epoch 4/10
103/103 [==============================] - 24s 229ms/step - loss: 0.0993 - acc: 0.9619 - val_loss: 0.1764 - val_acc: 0.9463
Epoch 5/10
103/103 [==============================] - 24s 228ms/step - loss: 0.0886 - acc: 0.9673 - val_loss: 0.1345 - val_acc: 0.9659
Epoch 6/10
103/103 [==============================] - 23s 228ms/step - loss: 0.0650 - acc: 0.9788 - val_loss: 0.1472 - val_acc: 0.9463
Epoch 7/10
103/103 [==============================] - 24s 229ms/step - loss: 0.0467 - acc: 0.9838 - val_loss: 0.1436 - val_acc: 0.9463
Epoch 8/10
103/103 [==============================] - 2

In [ ]:
model.save('model.h5')

In [ ]:
X_test = np.array(X_test) / 255

In [ ]:
pred=model.predict(X_test)
predictions=[]
for i in pred:
  if i > [0.5]:
    predictions.append(1)
  else:
    predictions.append(0)

In [ ]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test,np.array(predictions))
print('Accuracy: {:.4f}'.format(ac))

Accuracy: 0.9593


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,np.array(predictions)))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       300
           1       0.97      0.95      0.96       315

    accuracy                           0.96       615
   macro avg       0.96      0.96      0.96       615
weighted avg       0.96      0.96      0.96       615



**ACCURACY = 96%**